In [1]:
import os
######################################################
import csv
import pickle
from copy import deepcopy

In [2]:
def readcounts_to_rpm(readcounts):
    for chrom in readcounts.keys():
        totalread = sum(readcounts[chrom])
        for position in range(len(readcounts[chrom])):
            readcounts[chrom][position]/=float(totalread)
            readcounts[chrom][position]*=1E6
            
def writecountsf(rpm,filepath): 
    import struct
    f2=open(filepath+"keys","w")
    for chrom in rpm.keys():
        f=open(filepath+"genome","wb")
        for position in rpm[chrom]:
            f.write(struct.pack("f",position))
        f.close()
        f2.write(chrom+"\n")
    f2.close()

def countstowig(rpm,filestring):
    import random
    f=open(filestring+".wig","w")
    filestring=filestring.partition("_")[0][-3:]

    random.seed(filestring)
    c1=random.randint(0,255)
    random.seed(filestring+"xxx")
    c2=random.randint(0,255)
    random.seed(filestring+"000")
    c3=random.randint(0,255)

    f.write("track name=tracklabel viewLimits=-5:5 color="+str(c1)+','+str(c2)+','+str(c3)+"\n")
    for chrom in rpm.keys():
        if chrom[0:3]=='chr':
            f.write("fixedStep  chrom="+chrom+"  start=1  step=1\n")
        else:
            f.write("fixedStep  chrom=\""+chrom+"\"  start=1  step=1\n")


        for position in rpm[chrom]:
            f.write(str(position)+"\n")
    f.close()

In [3]:
def five_end_mapping_stallreporter(samgen,fasta):
    from BCBio import GFF
    from Bio import Seq
    outputdata1={}
    outputdata2={}
    #######################################################
    with open(fasta,'r') as f:
        sequence = ''
        for line in f:
            if line[0] == '>':
                continue
            sequence = sequence+line
    #######################################################    
    outputdata1['plus']=[0 for x in range(len(sequence))]
    outputdata2['minus']=[0 for x in range(len(sequence))]
    
    for read in samgen:
        if read[0][0] == '@':   
            continue
        if read[1] == '4':      
            continue            
        chrom = read[2]             
        readid = read[0]            
        startp = int(read[3]) - 1    
        seq = Seq.Seq(read[9])      
        length = len(seq)           
        if (read[1] == '16'):
            start = startp + length - 1
            outputdata2['minus'][start] += 1 ##### 
        if (read[1] == '0'):
            start = startp
            outputdata1['plus'][start] += 1 ##### 
    return [outputdata1,outputdata2] 

In [4]:
filepath = ''
fastq_path = filepath+'fastq_5RACE/'
fasta_path = filepath+'StallReporters/'

files = ['ks'+str(x) for x in range(222,226)]+['ks'+str(x) for x in range(230,234)]

# cutadapt
cutadapt_input_extension = '.fastq'
cutadapt_output_extension = '-ctadpt.fastq'
cutadapt_adapter = 'TGCCCGAGTG'
cutadapt_option = '--minimum-length 11 --discard-untrimmed'
cutadapt_output_extension_3end = '-ctadpt_ctadpt3.fastq'
cutadapt_adapter_3end = 'GCGGTCGAGTTCTGGACCGA'
cutadapt_option_3end = '--minimum-length 11'

# bowtie
inpath = fastq_path
outpath = filepath
bowtie_input_extension = cutadapt_output_extension_3end

# mapping
mapping = '_5map'

In [5]:
log_file = filepath+'log_5RACE_cutadapt.txt'
###
logs = open(log_file, 'w')
logs.close()
###

for f in files:
    print f
    ###
    cutadapt_inputfile = fastq_path+f+cutadapt_input_extension
    cutadapt_outputfile = fastq_path+f+cutadapt_output_extension

    command = 'cutadapt -g ^%s %s -o %s %s >> %s' % (
        cutadapt_adapter,
        cutadapt_option, 
        cutadapt_outputfile,
        cutadapt_inputfile,
        log_file
        )
    
    logs = open(log_file, 'a+')
    logs.write('\n\n'+f+'\n')
    logs.close()
    
    os.system(command)
    ###

print 
for line in open(log_file, 'r'):
    print line[:-1]

ks222
ks223
ks224
ks225
ks230
ks231
ks232
ks233



ks222
This is cutadapt 1.16 with Python 2.7.15
Command line parameters: -g ^TGCCCGAGTG --minimum-length 11 --discard-untrimmed -o fastq_5RACE/ks222-ctadpt.fastq fastq_5RACE/ks222.fastq
Running on 1 core
Trimming 1 adapter with at most 10.0% errors in single-end mode ...
Finished in 0.31 s (8 us/read; 7.26 M reads/minute).

=== Summary ===

Total reads processed:                  37,788
Reads with adapters:                    33,175 (87.8%)
Reads that were too short:                   0 (0.0%)
Reads written (passing filters):        33,175 (87.8%)

Total basepairs processed:     5,705,988 bp
Total written (filtered):      4,678,812 bp (82.0%)

=== Adapter 1 ===

Sequence: TGCCCGAGTG; Type: anchored 5'; Length: 10; Trimmed: 33175 times.

No. of allowed errors:
0-9 bp: 0; 10 bp: 1

Overview of removed sequences
length	count	expect	max.err	error counts
9	1490	0.1	0	0 1490
10	31332	0.0	1	30779 553
11	353	0.0	1	0 353



ks223
This is cutadap

In [6]:
log_file = filepath+'log_5RACE_cutadapt_3end.txt'
###
logs = open(log_file, 'w')
logs.close()
###

for f in files:
    print f
    ###
    cutadapt_inputfile = fastq_path+f+cutadapt_output_extension
    cutadapt_outputfile = fastq_path+f+cutadapt_output_extension_3end

    command = 'cutadapt -a %s %s -o %s %s >> %s' % (
        cutadapt_adapter_3end,
        cutadapt_option_3end, 
        cutadapt_outputfile,
        cutadapt_inputfile,
        log_file
        )
    
    logs = open(log_file, 'a+')
    logs.write('\n\n'+f+'\n')
    logs.close()
    
    os.system(command)
    ###

print 
for line in open(log_file, 'r'):
    print line[:-1]

ks222
ks223
ks224
ks225
ks230
ks231
ks232
ks233



ks222
This is cutadapt 1.16 with Python 2.7.15
Command line parameters: -a GCGGTCGAGTTCTGGACCGA --minimum-length 11 -o fastq_5RACE/ks222-ctadpt_ctadpt3.fastq fastq_5RACE/ks222-ctadpt.fastq
Running on 1 core
Trimming 1 adapter with at most 10.0% errors in single-end mode ...
Finished in 0.31 s (9 us/read; 6.39 M reads/minute).

=== Summary ===

Total reads processed:                  33,175
Reads with adapters:                    29,370 (88.5%)
Reads that were too short:               3,876 (11.7%)
Reads written (passing filters):        29,299 (88.3%)

Total basepairs processed:     4,678,812 bp
Total written (filtered):      2,223,331 bp (47.5%)

=== Adapter 1 ===

Sequence: GCGGTCGAGTTCTGGACCGA; Type: regular 3'; Length: 20; Trimmed: 29370 times.

No. of allowed errors:
0-9 bp: 0; 10-19 bp: 1; 20 bp: 2

Bases preceding removed adapters:
  A: 98.6%
  C: 0.2%
  G: 0.1%
  T: 0.1%
  none/other: 1.0%
    The adapter is preceded by "A" ext

In [7]:
######################################################

files_reporter = ['ks'+str(x) for x in range(222,226)]
stall_reporter = 'secM'

######################################################

###
path_stallreporter = outpath + 'alignments/StallReporter_'+stall_reporter+'/'
if not os.path.exists(outpath + 'alignments/'):
    os.makedirs(outpath + 'alignments/')
if not os.path.exists(path_stallreporter):
    os.makedirs(path_stallreporter)
###
log_file = filepath+'log_5RACE_Bowtie_StallReporter_'+stall_reporter+'.txt'
logs = open(log_file, 'w')
logs.close()
###     
        
for fname in files_reporter:
    print fname

    bowtie_command = 'bowtie -v 2 -y -m 3 -a --best --strata -S -p 30 '\
        '--un %s%s --max %s%s --al %s%s %s %s%s %s%s 1>> %s 2>> %s'
     
    bowtie_stallreporter = bowtie_command %(
        path_stallreporter,
        fname+'_nomatch.fastq',
        path_stallreporter,
        fname+'_multi.fastq',
        path_stallreporter,
        fname+'_match.fastq',
        fasta_path+'StallReporter_'+stall_reporter,
        inpath,
        fname + bowtie_input_extension,
        path_stallreporter,
        fname + '_StallReporter_'+stall_reporter+'_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n\n'+fname+'\n'+bowtie_stallreporter+'\n')
    logs.close()
    ###
    os.system(bowtie_stallreporter)           

print 
print 
for line in open(log_file, 'r'):
    print line[:-1]

ks222
ks223
ks224
ks225




ks222
bowtie -v 2 -y -m 3 -a --best --strata -S -p 30 --un alignments/StallReporter_secM/ks222_nomatch.fastq --max alignments/StallReporter_secM/ks222_multi.fastq --al alignments/StallReporter_secM/ks222_match.fastq StallReporters/StallReporter_secM fastq_5RACE/ks222-ctadpt_ctadpt3.fastq alignments/StallReporter_secM/ks222_StallReporter_secM_match.SAM 1>> log_5RACE_Bowtie_StallReporter_secM.txt 2>> log_5RACE_Bowtie_StallReporter_secM.txt
# reads processed: 29299
# reads with at least one reported alignment: 25177 (85.93%)
# reads that failed to align: 4122 (14.07%)
Reported 25184 alignments to 1 output stream(s)


ks223
bowtie -v 2 -y -m 3 -a --best --strata -S -p 30 --un alignments/StallReporter_secM/ks223_nomatch.fastq --max alignments/StallReporter_secM/ks223_multi.fastq --al alignments/StallReporter_secM/ks223_match.fastq StallReporters/StallReporter_secM fastq_5RACE/ks223-ctadpt_ctadpt3.fastq alignments/StallReporter_secM/ks223_StallReporter_secM_match.

In [8]:
fasta = fasta_path+'StallReporter_'+stall_reporter+'.fasta'

out_path=filepath+'density_StallReporter/'
if not os.path.exists(out_path):
    os.makedirs(out_path)

for f in files_reporter:
    samgenmain_filename=filepath+'alignments/StallReporter_'+stall_reporter+'/'+f+'_StallReporter_'+stall_reporter+'_match.SAM'
    f_samgenmain=open(samgenmain_filename)
    samgenmain=csv.reader(f_samgenmain,delimiter='\t')

    three_mapped=five_end_mapping_stallreporter(samgenmain,fasta)
    ###
    pickle_readcounts = out_path+f+stall_reporter+mapping+'_readcounts.pickle'
    with open(pickle_readcounts, 'wb') as readcounts_f:
        pickle.dump(three_mapped,readcounts_f)
    ###
    three_mapped_rpm = deepcopy(three_mapped)
    readcounts_to_rpm(three_mapped_rpm[0])
    #readcounts_to_rpm(three_mapped_rpm[1])
    pickle_rpm = out_path+f+stall_reporter+mapping+'_rpm.pickle'
    with open(pickle_rpm, 'wb') as rpm_f:
        pickle.dump(three_mapped_rpm,rpm_f)

    density_path = out_path+f+stall_reporter+mapping
    writecountsf(three_mapped_rpm[0],density_path+"_plus_")
    countstowig(three_mapped_rpm[0],density_path+"_plus")
    writecountsf(three_mapped_rpm[1],density_path+"_minus_")
    countstowig(three_mapped_rpm[1],density_path+"_minus")

In [9]:
######################################################

files_reporter = ['ks'+str(x) for x in range(230,234)]
stall_reporter = 'EPstop'

######################################################

###
path_stallreporter = outpath + 'alignments/StallReporter_'+stall_reporter+'/'
if not os.path.exists(outpath + 'alignments/'):
    os.makedirs(outpath + 'alignments/')
if not os.path.exists(path_stallreporter):
    os.makedirs(path_stallreporter)
###
log_file = filepath+'log_5RACE_Bowtie_StallReporter_'+stall_reporter+'.txt'
logs = open(log_file, 'w')
logs.close()
###     
        
for fname in files_reporter:
    print fname

    bowtie_command = 'bowtie -v 2 -y -m 3 -a --best --strata -S -p 30 '\
        '--un %s%s --max %s%s --al %s%s %s %s%s %s%s 1>> %s 2>> %s'
     
    bowtie_stallreporter = bowtie_command %(
        path_stallreporter,
        fname+'_nomatch.fastq',
        path_stallreporter,
        fname+'_multi.fastq',
        path_stallreporter,
        fname+'_match.fastq',
        fasta_path+'StallReporter_'+stall_reporter,
        inpath,
        fname + bowtie_input_extension,
        path_stallreporter,
        fname + '_StallReporter_'+stall_reporter+'_match.SAM',
        log_file,
        log_file
        )
    ###
    logs = open(log_file, 'a+')
    logs.write('\n\n'+fname+'\n'+bowtie_stallreporter+'\n')
    logs.close()
    ###
    os.system(bowtie_stallreporter)           

print 
print 
for line in open(log_file, 'r'):
    print line[:-1]

ks230
ks231
ks232
ks233




ks230
bowtie -v 2 -y -m 3 -a --best --strata -S -p 30 --un alignments/StallReporter_EPstop/ks230_nomatch.fastq --max alignments/StallReporter_EPstop/ks230_multi.fastq --al alignments/StallReporter_EPstop/ks230_match.fastq StallReporters/StallReporter_EPstop fastq_5RACE/ks230-ctadpt_ctadpt3.fastq alignments/StallReporter_EPstop/ks230_StallReporter_EPstop_match.SAM 1>> log_5RACE_Bowtie_StallReporter_EPstop.txt 2>> log_5RACE_Bowtie_StallReporter_EPstop.txt
# reads processed: 31562
# reads with at least one reported alignment: 25750 (81.59%)
# reads that failed to align: 5812 (18.41%)
Reported 25751 alignments to 1 output stream(s)


ks231
bowtie -v 2 -y -m 3 -a --best --strata -S -p 30 --un alignments/StallReporter_EPstop/ks231_nomatch.fastq --max alignments/StallReporter_EPstop/ks231_multi.fastq --al alignments/StallReporter_EPstop/ks231_match.fastq StallReporters/StallReporter_EPstop fastq_5RACE/ks231-ctadpt_ctadpt3.fastq alignments/StallReporter_EPstop/ks231

In [10]:
fasta = fasta_path+'StallReporter_'+stall_reporter+'.fasta'

out_path=filepath+'density_StallReporter/'
if not os.path.exists(out_path):
    os.makedirs(out_path)

for f in files_reporter:
    samgenmain_filename=filepath+'alignments/StallReporter_'+stall_reporter+'/'+f+'_StallReporter_'+stall_reporter+'_match.SAM'
    f_samgenmain=open(samgenmain_filename)
    samgenmain=csv.reader(f_samgenmain,delimiter='\t')

    three_mapped=five_end_mapping_stallreporter(samgenmain,fasta)
    ###
    pickle_readcounts = out_path+f+stall_reporter+mapping+'_readcounts.pickle'
    with open(pickle_readcounts, 'wb') as readcounts_f:
        pickle.dump(three_mapped,readcounts_f)
    ###
    three_mapped_rpm = deepcopy(three_mapped)
    readcounts_to_rpm(three_mapped_rpm[0])
    #readcounts_to_rpm(three_mapped_rpm[1])
    pickle_rpm = out_path+f+stall_reporter+mapping+'_rpm.pickle'
    with open(pickle_rpm, 'wb') as rpm_f:
        pickle.dump(three_mapped_rpm,rpm_f)

    density_path = out_path+f+stall_reporter+mapping
    writecountsf(three_mapped_rpm[0],density_path+"_plus_")
    countstowig(three_mapped_rpm[0],density_path+"_plus")
    writecountsf(three_mapped_rpm[1],density_path+"_minus_")
    countstowig(three_mapped_rpm[1],density_path+"_minus")